### GNS3 REST API

Based on examples from the GNS3 Docs pages.

In [1]:
import pandas as pd
import requests

In [2]:
r = requests.get("http://localhost:3080/v2/projects")
print(r.status_code)
#print(r.text)

200


In [3]:
#r.json()

We can get a list of projects in the current profile.

In [13]:
df = pd.read_json(r.text)
df.shape

(4, 18)

In [14]:
df.head()

auto_close  auto_open  auto_start       filename  grid_size      name  \
0        True      False       False   sandbox.gns3          0   sandbox   
1        True      False       False    splunk.gns3          0    splunk   
2        True      False       False  untitled.gns3          0  untitled   
3        True      False       False     kevin.gns3          0     kevin   

                                  path                            project_id  \
0   /Users/kevin/GNS3/projects/sandbox  e6099621-b13f-4605-ba61-393cfcf7ceb8   
1    /Users/kevin/GNS3/projects/splunk  51ee30ed-3f8d-467e-8990-e042b0e1c0fe   
2  /Users/kevin/GNS3/projects/untitled  e4a6165e-32cc-499a-99e5-ff0e08443606   
3     /Users/kevin/GNS3/projects/kevin  bcc63fd0-a205-440d-8872-e5fb604adac2   

   scene_height  scene_width  show_grid  show_interface_labels  show_layers  \
0          1000         2000      False                   True        False   
1          1000         2000      False                   True        False   
2          1000         2000      False                   True        False   
3          1000         2000      False                   True        False   

   snap_to_grid  status  supplier  variables  zoom  
0         False  closed       NaN        NaN   100  
1         False  closed       NaN        NaN   100  
2         False  opened       NaN        NaN   100  
3         False  closed       NaN        NaN   100

OK, so `read_json` handles list by creating separate rows in the DataFrame.  

In [15]:
df[['name','project_id']]

name                            project_id
0   sandbox  e6099621-b13f-4605-ba61-393cfcf7ceb8
1    splunk  51ee30ed-3f8d-467e-8990-e042b0e1c0fe
2  untitled  e4a6165e-32cc-499a-99e5-ff0e08443606
3     kevin  bcc63fd0-a205-440d-8872-e5fb604adac2

I'd like to look at what `nodes` are defined in a particular project - here we are looking at the csr-test project.

In [20]:
rproj = requests.get("http://localhost:3080/v2/projects/e4a6165e-32cc-499a-99e5-ff0e08443606")

In [19]:
print(rproj.text)

{
    "auto_close": true,
    "auto_open": false,
    "auto_start": false,
    "filename": "untitled.gns3",
    "grid_size": 0,
    "name": "untitled",
    "path": "/Users/kevin/GNS3/projects/untitled",
    "project_id": "e4a6165e-32cc-499a-99e5-ff0e08443606",
    "scene_height": 1000,
    "scene_width": 2000,
    "show_grid": false,
    "show_interface_labels": true,
    "show_layers": false,
    "snap_to_grid": false,
    "status": "opened",
    "supplier": null,
    "variables": null,
    "zoom": 100
}


In [21]:
import json
projectInfo=json.loads(rproj.text)

In [22]:
print(projectInfo["name"])

untitled


In [23]:
rnodes = requests.get("http://localhost:3080/v2/projects/e4a6165e-32cc-499a-99e5-ff0e08443606/nodes")

In [24]:
dfnodes= pd.read_json(rnodes.text)
dfnodes
#print(rnodes.text)

command_line compute_id  console console_host console_type  first_port_name  \
0                   local     5000    127.0.0.1       telnet              NaN   

   height                                              label  name  \
0      59  {'rotation': 0, 'style': 'font-family: TypeWri...  PC-1   

                                      node_directory  ... port_segment_size  \
0  /Users/kevin/GNS3/projects/untitled/project-fi...  ...                 0   

                                               ports  \
0  [{'adapter_number': 0, 'data_link_types': {'Et...   

                             project_id  properties   status  \
0  e4a6165e-32cc-499a-99e5-ff0e08443606          {}  stopped   

                     symbol width    x  y  z  
0  :/symbols/vpcs_guest.svg    65 -108 -5  0  

[1 rows x 23 columns]

## Stop a Node

with project and node id's, we can make a call to stop a node.  This is a POST, compared to the GET's we have been doing so far.

In [35]:
rs = requests.post("http://localhost:3080/v2/projects/20ee663c-602b-4b37-9c29-929c03ca3d99/nodes/48999888-0c1f-4996-a635-1debcd21e855/stop")